In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Cargar el dataset
file_path = "../data/processed/train_data_processed_imputeKnn_scale.csv"
data = pd.read_csv(file_path)

# Inspección inicial
print("Dimensiones del dataset antes de eliminar atípicos:", data.shape)
print(data.head())

# Identificar la variable objetivo y las características
target_col = "target"
features = [col for col in data.columns if col != target_col and col != 'therapeutic_area']  # Excluir 'therapeutic_area'

# Codificar columnas categóricas
label_encoders = {}  # Almacenar los codificadores para revertir la codificación si es necesario
for col in features:
    if data[col].dtype == 'object':  # Si la columna es de tipo objeto (categórica)
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le  # Guardar el codificador para futuras referencias

# Función para eliminar valores atípicos usando el rango intercuartílico (IQR)
def remove_outliers(df, cols):
    for col in cols:
        if np.issubdtype(df[col].dtype, np.number):  # Solo para columnas numéricas
            Q1 = df[col].quantile(0.25)  # Primer cuartil
            Q3 = df[col].quantile(0.75)  # Tercer cuartil
            IQR = Q3 - Q1  # Rango intercuartílico
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Eliminar valores atípicos por área terapéutica
therapeutic_areas = data['therapeutic_area'].unique()
data_cleaned = pd.DataFrame()  # Para almacenar los datos sin atípicos

for area in therapeutic_areas:
    area_data = data[data['therapeutic_area'] == area]
    area_data = remove_outliers(area_data, features)  # Eliminar atípicos
    data_cleaned = pd.concat([data_cleaned, area_data], ignore_index=True)

print("Dimensiones del dataset después de eliminar atípicos:", data_cleaned.shape)

# Entrenamiento de modelos para cada área terapéutica
models_by_area = {}
rmse_by_area = {}

for area in therapeutic_areas:
    print(f"\nEntrenando modelo para el área terapéutica: {area}")
    
    # Filtrar los datos por área terapéutica
    area_data = data_cleaned[data_cleaned['therapeutic_area'] == area]
    X_area = area_data[features]
    y_area = area_data[target_col]
    
    # Dividir en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X_area, y_area, test_size=0.2, random_state=42)

    # Definir el modelo XGBoost
    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

    # Definir el espacio de búsqueda de hiperparámetros
    param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [50, 100, 200],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 0.1, 0.2],
        'min_child_weight': [1, 5, 10]
    }

    # Realizar la búsqueda de hiperparámetros con validación cruzada
    grid_search = GridSearchCV(estimator=xgb_model,
                               param_grid=param_grid,
                               scoring='neg_mean_squared_error', 
                               cv=3, n_jobs=-1, verbose=2)
    
    grid_search.fit(X_train, y_train)

    # Mostrar los mejores parámetros
    print("Mejores hiperparámetros para el área:", area)
    print(grid_search.best_params_)

    # Usar el mejor modelo encontrado
    best_xgb_model = grid_search.best_estimator_

    # Realizar predicciones en el conjunto de validación
    y_pred = best_xgb_model.predict(X_val)
    
    # Calcular el RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_by_area[area] = rmse
    
    # Almacenar el modelo entrenado
    models_by_area[area] = best_xgb_model
    
    print(f"RMSE para el área {area}: {rmse:.4f}")

# Resumen del RMSE por área terapéutica
print("\nResumen de RMSE por área terapéutica:")
for area, rmse in rmse_by_area.items():
    print(f"Área {area}: RMSE = {rmse:.4f}")

# Guardar los modelos entrenados en disco
import joblib

for area, model in models_by_area.items():
    model_filename = f"xgb_model_{area}.joblib"
    joblib.dump(model, model_filename)
    print(f"Modelo para el área {area} guardado como {model_filename}")


In [ ]:
# Cargar el dataset de submission
submission_file_path = "../data/processed/submission_data_processed_imputeKnn_scale.csv"
submission_data = pd.read_csv(submission_file_path)
submission_data_raw = pd.read_csv("../data/raw/submission_data.csv")

# Asegurar que las características coincidan con las usadas durante el entrenamiento
assert set(features).issubset(submission_data.columns), "Las características no coinciden entre train y submission"

# Codificar las columnas categóricas en submission_data usando los LabelEncoders entrenados
for col, le in label_encoders.items():
    if col in submission_data.columns:
        # Verificar si hay categorías nuevas
        submission_data[col] = submission_data[col].apply(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1  # Asignar -1 a valores desconocidos
        )

# Lista para almacenar las predicciones
submission_predictions = []

# Iterar sobre cada fila del dataset de submission
for _, row in submission_data.iterrows():
    # Identificar el área terapéutica de la fila
    therapeutic_area = row['therapeutic_area']
    
    # Seleccionar el modelo correspondiente
    if therapeutic_area in models_dict:
        model = models_dict[therapeutic_area]
        
        # Seleccionar las características relevantes para predicción
        row_features = row[features].values.reshape(1, -1)  # Convertir en un array 2D
        
        # Realizar la predicción
        prediction = model.predict(row_features)[0]  # Extraer el valor predicho
    else:
        # Asignar NaN si no hay modelo para el área terapéutica
        prediction = np.nan
        print(f"Advertencia: No se encontró modelo para el área terapéutica '{therapeutic_area}'. Se asigna NaN.")

    # Almacenar la predicción
    submission_predictions.append(prediction)

# Agregar las predicciones como una nueva columna en el DataFrame de submission
submission_data['prediction'] = submission_predictions
submission_data['cluster_nl'] = submission_data_raw['cluster_nl']
submission_data['date'] = submission_data_raw['date']

submission_data = submission_data[['date', 'cluster_nl', 'prediction']]

# Guardar las predicciones en un archivo CSV
submission_output_path = "../data/processed/submission_predictions.csv"
submission_data.to_csv(submission_output_path, index=False)
print(f"Predicciones para submission guardadas en: {submission_output_path}")


In [ ]:
# Cargar el dataset de submission
submission_file_path = "../data/processed/submission_data_processed_imputeKnn_scale.csv"
submission_data = pd.read_csv(submission_file_path)

# Asegurar que las características coincidan con las usadas durante el entrenamiento
assert set(features).issubset(submission_data.columns), "Las características no coinciden entre train y submission"

# Codificar las columnas categóricas en submission_data usando los LabelEncoders entrenados
for col, le in label_encoders.items():
    if col in submission_data.columns:
        # Verificar si hay categorías nuevas
        submission_data[col] = submission_data[col].apply(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1  # Asignar -1 a valores desconocidos
        )


# Lista para almacenar las predicciones
submission_predictions = []

# Iterar sobre cada fila del dataset de submission
for _, row in submission_data.iterrows():
    # Identificar el área terapéutica de la fila
    therapeutic_area = row['therapeutic_area']
    
    # Seleccionar el modelo correspondiente
    if therapeutic_area in models_dict:
        model = models_dict[therapeutic_area]
        
        # Seleccionar las características relevantes para predicción
        row_features = row[features].values.reshape(1, -1)  # Convertir en un array 2D
        
        # Realizar la predicción
        prediction = model.predict(row_features)[0]  # Extraer el valor predicho
    else:
        # Asignar NaN si no hay modelo para el área terapéutica
        prediction = np.nan
        print(f"Advertencia: No se encontró modelo para el área terapéutica '{therapeutic_area}'. Se asigna NaN.")

    # Almacenar la predicción
    submission_predictions.append(prediction)

# Agregar las predicciones como una nueva columna en el DataFrame de submission
submission_data['prediction'] = submission_predictions

# Guardar las predicciones en un archivo CSV
submission_output_path = "../data/processed/submission_predictions.csv"
submission_data.to_csv(submission_output_path, index=False)
print(f"Predicciones para submission guardadas en: {submission_output_path}")